In [1]:
from read_datasetBreakfast import load_data, read_mapping_dict, load_data_cust
import os
import torch
import seaborn as sn

COMP_PATH = 'C:\\Users\\cherl\\Desktop\\Masters\\CS5242 Neural Networks and Deep Learning\\Project'
''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)
if  split == 'training':
    data_feat_train, data_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
    data_feat_train_cust, data_labels_cust = load_data_cust(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
if  split == 'test':
    data_feat_test = load_data(test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only

#'''
#Write Code Below
#Pointers
#Need to load the segments.txt file for segments for test videos 
#Output the CSV in correct format as shown in Evaluation Section
#Id corresponds to the segments in order. 
#Example - 30-150 = Id 0
#          150-428 = Id 1
#          428-575 = Id 2
#Category is the Class of the Predicted Action
#'''

Finish Load the Training data and labels!!!
Finish Load the Training data and labels!!!


In [9]:
#Print info on the data
print("Number of Training Data: {}".format(len(data_feat_train)))
print("Number of Training Data Labels: {}".format(len(data_labels)))
print("Size of Training Data #0: {}".format(data_feat_train[0].size()))
#print(data_feat_train[0][0])
print(len(data_labels_cust[0]))
#Testing Data
try:
    print("Number of Test Data: {}".format(len(data_feat_test)))
    print("% of Training Data: {}".format(len(data_feat_test)/(len(data_feat_train)+len(data_feat_test))*100))
except NameError:
    print("Testing Data not loaded")

Number of Training Data: 28
Number of Training Data Labels: 28
Size of Training Data #0: torch.Size([743, 400])
743
Testing Data not loaded


In [12]:
import torch.nn as nn
import torch.nn.functional as F

#Define the neural network module
class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, stride=1, padding=0).double()
        self.pool = nn.MaxPool2d((2, 2), stride=1).double()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        return x

In [16]:
# Helper functions to run the training
def cal_acc(probs, target):
    # probs: probability that each image is labeled as 1
    # target: ground truth label
    with torch.no_grad():
      prediction = torch.argmax(probs, axis=-1)    
      acc = torch.sum(prediction == target)
    return acc.item() / len(target) * 100

def train_one_pass(network, optim, device):
  # Input: Network and Optimizer
  # Output: Averge accuracy , Avergae loss in the pass
  network.train()
  acc_one_pass = []
  loss_one_pass = []
  for i, segment in enumerate(data_feat_train):
    X = segment.reshape(segment.size()[0], 20, 20).unsqueeze(dim=1)
    X = torch.tensor(X).to(device)
    y = torch.tensor(data_labels_cust[i]).to(device)
    predy = network(X)
    #probs = F.softmax(outputs, dim=1)
    #predy = torch.argmax(probs, dim=1)
    cross_entrophy_loss = nn.CrossEntropyLoss()
    loss = cross_entrophy_loss(predy, y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i % 100 == 0:
          print("Iter: {:.2f}. ".format(i / len(trainloader)),
            "Train Total Loss: {:.2f}. ".format(loss.item()),
            "Train Accuracy: {:.2f}% ".format(cal_acc(predy, y))
          )
    acc_one_pass.append(cal_acc(predy, y))
    loss_one_pass.append(loss.item())
    return sum(acc_one_pass) / len(acc_one_pass), sum(loss_one_pass) / len(loss_one_pass)

In [17]:
#Run the training

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

my_cnn = mynet().to(device)
optimizer = torch.optim.SGD(my_cnn.parameters(), lr=0.1)
    
avg_acc, avg_loss = train_one_pass(my_cnn, optimizer, device)

C:\Users\cherl\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: 1only batches of spatial targets supported (non-empty 3D tensors) but got targets of size: : [743]